<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/tidy_up_preprocessing_notebook/notebooks/processed/ct_preprocessing_ubs_pairing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
"""
===================================================
Author: Chiaki Tachikawa
Role: Data Science Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/chiaki-tachikawa
Date: 2025-02-13
Version: 1.1

Description:
    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:
    - Importing necessary libraries and downloading NLTK data.
    - Defining and applying a `preprocessor` function to clean and tokenize text data.
    - Reading and preprocessing various CSV files containing transcript data.
    - Exporting the preprocessed data to new CSV files for further analysis.

===================================================
"""

'\n===================================================\nAuthor: Chiaki Tachikawa\nRole: Data Science Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/chiaki-tachikawa\nDate: 2025-02-13\nVersion: 1.1\n\nDescription:\n    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:\n    - Importing necessary libraries and downloading NLTK data.\n    - Defining and applying a `preprocessor` function to clean and tokenize text data.\n    - Reading and preprocessing various CSV files containing transcript data.\n    - Exporting the preprocessed data to new CSV files for further analysis.\n\n===================================================\n'

# **Library**

In [80]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from google.colab import drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Function**

preprocessor function : The function modifies the DataFrame data in place, adding two new columns (col1 and col2) with preprocessed text.


Input:
  - name of dataframe
  - name of column which contains the text to clean
  - name of column which is tokenized
  - name of column which is cleaned

In [81]:
#create function to preprocess data
def preprocessor (data, col, col1,col2):
  #Copy col1umn
  data[col1]=data[col]
  data[col2]=data[col]


  #Adding column1
  #Lower the lettercase
  data[col1] = data[col1].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col1] = data[col1].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col1] = data[col1].apply(nltk.word_tokenize)

  #Remove numbers
  data[col1] = data[col1].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col1] = data[col1].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col1] = data[col1].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col1] = data[col1].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])

  #lemmatization
  lemmatizer = WordNetLemmatizer()
  data[col1] = data[col1].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])



  #Adding column2
  #Lower the lettercase
  data[col2] = data[col2].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col2] = data[col2].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #remove symbols
  data[col2] = data[col2].apply (lambda x: [re.sub(r"[.,'?]", "", x)])

  return


find_row: This function searches upwards from the given current_row_num in the DataFrame df to find the first row where the value in column "M" is "A". It returns the index of that row. If no such row is found, it returns 0

In [82]:
def find_row (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] == "A":
      break
    else:
      i-=1
  return i

find_row_empty: This function searches upwards from the given current_row_num to find the first row where col1 has the value "A" and col2 is not an empty list. It returns the index of that row.

In [83]:
def find_row_empty (df, col1, col2, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col1][i] == "A" and df[col2][i] != []:
      break
    else:
      i-=1
  return i

In [84]:
def create_ques_num_column (data, new_col,marker_col):
  #Create question number column
  data[new_col]=None
  #set global var to count question number
  num = 0
  #if Q was found, num adds 1 otherwise none
  for i in data.index:
    if data.loc[i,marker_col] is not np.NaN:
      data.at[i,new_col]=num
      num +=1
    else:
      continue

In [85]:
# Function to extract names
def extract_name(full_string):
    return full_string.split(',')[0]

In [86]:
#check if there is "A" before "Q" from the current location
def find_last_a (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] =="UBS":
      j = i-1
      while j > 0:
        if df[col][j]!="UBS":
          pass
        else:
          break
        j-=1
      break
    else:
      i-=1
  return i

## **Data**

In [87]:
#drive.mount('/content/drive')

In [88]:
#!ls"/content/bank_of_england/data/preprocessed_data/Archived/jpmorgan_qa_section_preprocessed.csv"

In [89]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


JP Morgan QA section

In [90]:
#Defining qa_data
qa_data = pd.read_csv("/content/ubs_data.csv")
qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"['Chis Hallam, Goldman']",NaN,Goldman
1,Sergio P. Ermotti,UBS,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,Goldman
2,Chris Hallam,Goldman,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"['Kian Abouhossein, JPMorgan']",NaN,Goldman
3,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,JPMorgan
4,Sergio P. Ermotti,UBS,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,JPMorgan


In [91]:
for i in range(len(qa_data)-1):
  qa_data.at[i,"ex_dummy"]=qa_data.at[i+1,"financial_quarter"]

In [92]:
qa_data[130:150]

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category
130,Adam Terelak,Mediobanca,Okay. Thank you.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Mediobanca
131,Andrew Coombs,Citigroup,Good morning it’s Andrew Coombs from Citi and ...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Citigroup
132,Todd Tuckner,UBS,"So, Andrew, in terms of – I’ll take the second...",31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Citigroup
133,Andrew Coombs,UBS,Thank you.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Citigroup
134,Vishal Shah,Morgan Stanley,Hi. Thank you so much for your questions. My f...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Morgan Stanley
135,Todd Tuckner,UBS,"Hey, Vishal. I mean I think on this on that se...",31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Morgan Stanley
136,Vishal Shah,Morgan Stanley,Okay. Thank you so much.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Morgan Stanley
137,Sergio P. Ermotti,UBS,Okay. The last answer and questions and I'm su...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q24,Morgan Stanley
138,Giulia Miotto,Morgan Stanley,Hi. Good morning. Thank you for taking my ques...,14 August 2024,2Q24,2q24-earnings-call-remarks.pdf,NaN,2Q24,Morgan Stanley
139,Todd Tuckner,UBS,"So, regarding the NII guidance in terms of the...",14 August 2024,2Q24,2q24-earnings-call-remarks.pdf,NaN,2Q24,Morgan Stanley


In [93]:
qa_data["question"] = None
qa_data["answer"] = None
current_bank=qa_data["category"][0]
question=""
answer=""
for i in range(len(qa_data)):
  if qa_data["ex_dummy"][i]!=qa_data["financial_quarter"][i] and qa_data["job_title"][i]=="UBS":
      answer += str(qa_data["utterance"][i])
      qa_data.at[i,"question"]=question
      qa_data.at[i,"answer"]=answer
      answer = ""
      question = ""
  else:
    if qa_data["job_title"][i]!="UBS" and current_bank==qa_data["category"][i]:
      question += str(qa_data["utterance"][i])
    elif qa_data["job_title"][i]=="UBS" and current_bank==qa_data["category"][i]:
      answer += str(qa_data["utterance"][i])
    elif qa_data["job_title"][i]!="UBS" and current_bank!=qa_data["category"][i]:
      current_bank=qa_data["job_title"][i]
      qa_data.at[i-1,"question"]=question
      qa_data.at[i-1,"answer"]=answer
      print()
      question = str(qa_data["utterance"][i])
      answer = ""
    else:
      continue


In [ ]:
"""
qa_data["question"]=None
qa_data["answer"] =None

indices_to_drop = []

for i in range(len(qa_data)):
  if qa_data["job_title"][i]!="UBS":
     qa_data.at[i,"question"] = qa_data.loc[i,"utterance"]
  else:
    if i >0 and tuple(qa_data.iloc[i-1])in[tuple(qa_data.iloc[i])]:  #qa_data.isin(qa_data[i-1]):
      qa_data.at[i-1,"answer"] = qa_data.loc[i,"utterance"]
    else:
      if i>1:
        qa_data.at[i-2,"answer"] = qa_data.loc[i,"utterance"]
    indices_to_drop.append(i)

qa_data.drop(indices_to_drop, inplace=True)
qa_data.reset_index(drop=True, inplace=True)
"""

'\nqa_data["question"]=None\nqa_data["answer"] =None\n\nindices_to_drop = []\n\nfor i in range(len(qa_data)):\n  if qa_data["job_title"][i]!="UBS":\n     qa_data.at[i,"question"] = qa_data.loc[i,"utterance"]\n  else:\n    if i >0 and tuple(qa_data.iloc[i-1])in[tuple(qa_data.iloc[i])]:  #qa_data.isin(qa_data[i-1]):\n      qa_data.at[i-1,"answer"] = qa_data.loc[i,"utterance"]\n    else:\n      if i>1:\n        qa_data.at[i-2,"answer"] = qa_data.loc[i,"utterance"]\n    indices_to_drop.append(i)\n\nqa_data.drop(indices_to_drop, inplace=True)\nqa_data.reset_index(drop=True, inplace=True)\n'

In [94]:
qa_data[130:150]

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category,question,answer
130,Adam Terelak,Mediobanca,Okay. Thank you.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Mediobanca,Morning. Thank you for the questions. I want t...,"Thanks, Adam. On the second one, would just sa..."
131,Andrew Coombs,Citigroup,Good morning it’s Andrew Coombs from Citi and ...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Citigroup,None,None
132,Todd Tuckner,UBS,"So, Andrew, in terms of – I’ll take the second...",31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Citigroup,None,None
133,Andrew Coombs,UBS,Thank you.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Citigroup,Good morning it’s Andrew Coombs from Citi and ...,"So, Andrew, in terms of – I’ll take the second..."
134,Vishal Shah,Morgan Stanley,Hi. Thank you so much for your questions. My f...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Morgan Stanley,None,None
135,Todd Tuckner,UBS,"Hey, Vishal. I mean I think on this on that se...",31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Morgan Stanley,None,None
136,Vishal Shah,Morgan Stanley,Okay. Thank you so much.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q23,Morgan Stanley,None,None
137,Sergio P. Ermotti,UBS,Okay. The last answer and questions and I'm su...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,NaN,2Q24,Morgan Stanley,Hi. Thank you so much for your questions. My f...,"Hey, Vishal. I mean I think on this on that se..."
138,Giulia Miotto,Morgan Stanley,Hi. Good morning. Thank you for taking my ques...,14 August 2024,2Q24,2q24-earnings-call-remarks.pdf,NaN,2Q24,Morgan Stanley,None,None
139,Todd Tuckner,UBS,"So, regarding the NII guidance in terms of the...",14 August 2024,2Q24,2q24-earnings-call-remarks.pdf,NaN,2Q24,Morgan Stanley,None,None


In [ ]:
filtered_data= qa_data.dropna(subset=["question"])

In [ ]:
filtered_data.reset_index(drop=True, inplace=True)

In [ ]:
filtered_data=filtered_data.drop(columns="ex_dummy")

In [ ]:
filtered_data.head(50)

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,category,question,answer
0,Chris Hallam,Goldman,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"['Kian Abouhossein, JPMorgan']",Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...","Okay. Thank you. On capital requirements, you ..."
1,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,JPMorgan,Yeah. Thanks. Just two questions. The first on...,"So, Sarah, take the first question. I'll take ..."
2,Alastair Ryan,Bank of America,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Bank of America,"Yeah. Thank you. Good morning. Welcome back, S...","Thank you, Ryan. It is good to be back to inte..."
3,Flora Bocahut,Jefferies,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Jefferies,Yes. Good morning. The first question I wanted...,"So, on the first question in terms of the tren..."
4,Andrew Coombs,Citi,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Citi,"Good morning. Two questions. Firstly, just on ...","So, on the first quarter or the first question..."
5,Adam Terelak,Mediobanca,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Mediobanca,Morning. I've got two. One is a bit of a follo...,So on the LCR and more generally the funding p...
6,Jeremy Sigee,Exane,"Great, thank you very much.",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"['Anke Reingen, RBC']",Exane,Morning. Thank you and welcome back to Sergio ...,"Thank you, Jeremy. Look, you know, the base pl..."
7,Sarah Youngwood,UBS,"Sure. So, in terms of the cost savings and EPS...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"['Amit Goel, Barclays']",RBC,Yeah. Thank you very much for taking my questi...,"So on the treasury share, what happened is we ..."
8,Sergio Ermotti,UBS,"Well, on the 25%, I mean, it depends how you m...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,Barclays,"Okay. Thanks. And just on the cost savings, is...","So, the 8 billion-dollars-plus was done based ..."
9,Tom Hallet,KBW,"Great. Thanks, Sergio.",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,KBW,"Good morning, everyone. So, just a couple for ...","Thank you, Tom. Now, look, of course, the reve..."


In [ ]:
filtered_data.to_csv("/content/ubs_qa_df_ver1.csv", index=False)

In [ ]:
"""
#ascend df by financial_quarter
quarter_order = ["1Q23", "2Q23","3Q23","4Q23","1Q24","2Q24","3Q24","4Q24"]
qa_data["financial_quarter"] = pd.Categorical(qa_data["financial_quarter"], categories=quarter_order, ordered=True)
qa_data = qa_data.sort_values("financial_quarter", kind="mergesort")
"""

'\n#ascend df by financial_quarter\nquarter_order = ["1Q23", "2Q23","3Q23","4Q23","1Q24","2Q24","3Q24","4Q24"]\nqa_data["financial_quarter"] = pd.Categorical(qa_data["financial_quarter"], categories=quarter_order, ordered=True)\nqa_data = qa_data.sort_values("financial_quarter", kind="mergesort")\n'

In [ ]:
#preprocessing data
preprocessor(qa_data, "utterance", "question_tokenised_data", "question_cleaned")
#preprocessor(qa_data,"utterance","answer_tokenised_data","answer_cleaned")

In [ ]:
#remove operater
#qa_data = qa_data.loc[qa_data["speaker"]!="Operator"]

#remove less than 20 words
qa_data["count"] = qa_data["question_tokenised_data"].apply(lambda x: len(x))
qa_data = qa_data.loc[qa_data["count"]>20]
qa_data.head()


,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category,question,answer,question_tokenised_data,question_cleaned,count
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"['Chis Hallam, Goldman']",NaN,Goldman,None,None,"[chi, hallam, goldman, sachs, yes, good, morni...",[chis hallam goldman sachs yes good morning ev...,96
1,Sergio P. Ermotti,UBS,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,Goldman,None,None,"[okay, thank, you, capital, requirement, know,...",[okay thank you capital requirements know situ...,73
3,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,JPMorgan,None,None,"[yeah, thanks, two, question, first, one, rela...",[yeah thanks two questions first one related b...,65
5,Sarah Youngwood,UBS,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"['Kian Abouhossein, JPMorgan']",NaN,JPMorgan,None,None,"[give, focused, intentionally, viewed, economi...",[so give 74% focused intentionally viewed econ...,27
9,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,JPMorgan,Yeah. Thanks. Just two questions. The first on...,"So, Sarah, take the first question. I'll take ...","[client, retention, maybe, let, reiterate, loo...",[client retention – maybe let reiterate look i...,80


In [ ]:
#reset index
qa_data.reset_index(drop=True, inplace=True)
qa_data.head(20)

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category,question,answer,question_tokenised_data,question_cleaned,count
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"['Chis Hallam, Goldman']",NaN,Goldman,None,None,"[chi, hallam, goldman, sachs, yes, good, morni...",[chis hallam goldman sachs yes good morning ev...,96
1,Sergio P. Ermotti,UBS,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,Goldman,None,None,"[okay, thank, you, capital, requirement, know,...",[okay thank you capital requirements know situ...,73
2,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,JPMorgan,None,None,"[yeah, thanks, two, question, first, one, rela...",[yeah thanks two questions first one related b...,65
3,Sarah Youngwood,UBS,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"['Kian Abouhossein, JPMorgan']",NaN,JPMorgan,None,None,"[give, focused, intentionally, viewed, economi...",[so give 74% focused intentionally viewed econ...,27
4,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,JPMorgan,Yeah. Thanks. Just two questions. The first on...,"So, Sarah, take the first question. I'll take ...","[client, retention, maybe, let, reiterate, loo...",[client retention – maybe let reiterate look i...,80
5,Alastair Ryan,Bank of America,"Yeah. Thank you. Good morning. Welcome back, S...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,Bank of America,None,None,"[yeah, thank, you, good, morning, welcome, bac...",[yeah thank you good morning welcome back serg...,74
6,Sergio P. Ermotti,UBS,"Thank you, Ryan. It is good to be back to inte...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,Bank of America,None,None,"[thank, you, ryan, good, back, interact, well,...",[thank you ryan good back interact well look y...,106
7,Flora Bocahut,Jefferies,Yes. Good morning. The first question I wanted...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,Jefferies,None,None,"[yes, good, morning, first, question, wanted, ...",[yes good morning first question wanted ask re...,96
8,Sergio P. Ermotti,UBS,"Yeah. On the client side. I mean, if you look ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,Jefferies,None,None,"[yeah, client, side, mean, look, example, asia...",[yeah client side mean look at example asia th...,146
9,Andrew Coombs,Citi,"Good morning. Two questions. Firstly, just on ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,NaN,NaN,Citi,None,None,"[good, morning, two, question, firstly, slide,...",[good morning two questions firstly slide 10 r...,111


In [ ]:
#rename column
qa_data.rename(columns={"utterance":"metadata_answer","speaker":"Exective", "analyst_title":"Bank"},inplace=True)
qa_data.drop(columns=["category"],inplace=True)

In [ ]:
#reorganise column
qa_data=qa_data[["source_file","financial_quarter","call_date","Exective","question_num","metadata_question","question_cleaned","metadata_answer", "answer_cleaned","analyst","Bank"]]

KeyError: "['question_cleaned', 'metadata_answer', 'answer_cleaned'] not in index"

# **Export the output as a csv file**

UBS QA section

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path3 = "/content/ubs_qa_df_preprocessed.csv"
qa_data.to_csv(preprocessed_qa_csv_path3, index=False)

UBS management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path4 = "/content/sample_data/ubs_management_df_preprocessed.csv"
ubs_manag_df.to_csv(preprocessed_qa_csv_path4, index=False)